In [110]:
#stdlib
import sys
import os
import json
# import warnings
# warnings.filterwarnings('ignore')


# relative import error with python modules and what not, the path needs to change according to cwd :)
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../.."))
if project_root not in sys.path:
    sys.path.append(project_root)
    # Load model directly

#hf
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from datasets import Dataset
from huggingface_hub import login

#pytorch
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# import numpy as np

#a2a
# import lib.embeddings as emb

In [ ]:
login(os.environ.get("HF_TOKEN"))

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [62]:
torch.cuda.empty_cache()

In [63]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [214]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


sentences = ['This is an example sentence yes.', 'Each sentence is converted yes.']

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# # Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
print(sentence_embeddings.shape)

# # Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2)
print(sentence_embeddings.shape)
# print("Sentence embeddings:")
# print(sentence_embeddings.shape)


torch.Size([2, 384])
torch.Size([2, 384])


In [198]:
model_output[0].shape, model_output[1].shape

(torch.Size([2, 9, 384]), torch.Size([2, 384]))

In [212]:
# encoded_input['attention_mask'].unsqueeze(-1).expand(model_output[0].size()).float()[1][0].shape
sentence_embeddings.shape

torch.Size([2, 384])

In [52]:
encoded_input['input_ids']

tensor([[ 101, 2023, 2003, 2019, 2742, 6251, 2748, 1012,  102],
        [ 101, 2169, 6251, 2003, 4991, 2748, 1012,  102,    0]])

In [123]:
# tokenizer.tokenize('abo 7ankal el sada7 may arrive at dawn.')
# tokenizer.tokenize('chicken bawarma alejandro sanchez al harbi')

['chicken',
 'ba',
 '##war',
 '##ma',
 'alejandro',
 'sanchez',
 'al',
 'ha',
 '##rb',
 '##i']

In [54]:
encoded_input.word_ids()


[None, 0, 1, 2, 3, 4, 5, 6, None]

In [113]:
tok2wrd = []
for wid in encoded_input.word_ids():
    if wid:
        a,b = encoded_input.word_to_tokens(wid)
        if a == b-1:
            x =[a]
        else:
            x = [a,b-1]
        if len(tok2wrd) == 0 or tok2wrd[-1] != x:
            tok2wrd.append(x)
# encoded_input.word_to_tokens()

In [112]:
encoded_input.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, None]

In [115]:
# Flatten the input IDs
word_ids = encoded_input["input_ids"].tolist()
flat_word_ids = [word_id for sentence in word_ids for word_id in sentence if word_id is not None]

# Convert IDs to tokens
tokens = tokenizer.convert_ids_to_tokens(flat_word_ids)
print("Tokens:", tokens)

Tokens: ['[CLS]', 'this', 'is', 'an', 'example', 'sentence', 'yes', '.', '[SEP]', '[CLS]', 'each', 'sentence', 'is', 'converted', 'yes', '.', '[SEP]', '[PAD]']


In [105]:
txt

'[unused0] [unused1] [unused2] [unused3] [unused4] [unused5]'

In [ ]:
model_output.keys(), model_output[0].shape, model_output[1].shape,

In [ ]:
encoded_input.keys(), encoded_input[0], encoded_input[1]

In [ ]:
from utils.misc import ftos
candidate = ftos("/home/turbouser44/research/Arxiv2Arxode/sandbox/code_6e04b216-f62b-4afb-93a1-e7fcabea0a8a.py")
ref = ftos("/home/turbouser44/research/Arxiv2Arxode/notebooks/reference_GAT.py")

In [ ]:
enc_in = tokenizer([candidate,ref], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    modelout = model(**enc_in)

# Perform pooling
embeddings = mean_pooling(modelout, enc_in['attention_mask'])

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings.shape)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# embeddings.shape
print(cosine_similarity(embeddings[0].reshape(1,-1), embeddings[1].reshape(1,-1)))
torch.dot(embeddings[0], embeddings[1])

In [ ]:
import transformers

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])
